In [57]:
# Created on:   03/03/2018  Aditya Shirode
# Modified on:  03/08/2018  Aditya Shirode

TO DO:
- Make generic functions for tasks for modularity
- CRON jobs for all timeframes
- One function to update all csvs
- Plug n play for indicators
- Add limit to queries

In [44]:
# Imports
import os
import time
import talib
import logging
import requests
import datetime
import importlib
import dateutil.parser

import numpy as np
import pandas as pd

from collections import defaultdict

In [2]:
importlib.reload(logging)
LOGGING_FORMAT = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(format=LOGGING_FORMAT, level=logging.INFO)

In [32]:
cryptocompare_wrapper = os.path.join(os.curdir, 'cryptocompare_wrapper.py')
import cryptocompare_wrapper as ccw

In [4]:
# FIELDS
PRICE = 'PRICE'
HIGH = 'HIGH24HOUR'
LOW = 'LOW24HOUR'
VOLUME = 'VOLUME24HOUR'
CHANGE = 'CHANGE24HOUR'
CHANGE_PERCENT = 'CHANGEPCT24HOUR'
MARKETCAP = 'MKTCAP'
NPERIODS = 100
TIMEFRAME = 'Day'

In [33]:
# Defaults
CURR = 'BTC'
EXCHANGE = 'CCCAGG'
COIN = 'ETH'
COIN_LIST = ['BTC', 'ETH', 'XRP']
EXCHANGES = ['Kucoin', 'Cryptopia', 'HitBTC']

In [6]:
# Coin DB
coins = ccw.get_coin_list()
COIN_DB = pd.DataFrame.from_dict(coins, orient='index')
# COIN_DB.to_csv('coin_list.csv')

In [7]:
# Exchange DB
exchanges = ccw.get_exchanges_list()
EXCHANGE_DB = pd.DataFrame.from_dict(exchanges, orient='index')
# EXCHANGE_DB.to_csv('exchanges_list.csv')

In [31]:
# Update historical data (per day) for all coins

csv_all_coins_day_full = 'all_coins_day_full.csv'
not_updated = defaultdict(list)
existing_coin_exchange = []
to_curr = 'BTC'

# If the csv already exists, find out which coins and exchanges have already been added
if os.path.isfile(csv_all_coins_day_full):
    df_csv_all_coins_day_full = pd.read_csv(csv_all_coins_day_full)
    # existing_coin_exchange is a list of tuples (coin, exchange)
    existing_coin_exchange = pd.MultiIndex.from_product((df_csv_all_coins_day_full['coin'].unique(),
                                                         df_csv_all_coins_day_full['exchange'].unique())).get_values()


for exchange in EXCHANGES :
    # for symbol in COIN_DB.Symbol:
    for symbol in COIN_LIST:
    # for symbol in EXCHANGE_DB.loc[exchange].dropna().index:
        # If the tuple does not exist in the csv
        if (symbol, exchange) not in existing_coin_exchange:
            try:
                # Can't fetch the same symbol in same symbol rate
                if symbol is not to_curr:
                    df_coin_day_all = ccw.get_historical_price_day_full(
                        coin=symbol,
                        to_curr=to_curr,
                        timestamp=time.time(),
                        exchange=exchange
                    )
                    if df_coin_day_all.empty:
                        not_updated[exchange].append(symbol)
                    else:
                        df_coin_day_all['exchange'] = exchange
                        df_coin_day_all['coin'] = symbol

                        # If csv does not exist, write, else append
                        if not os.path.isfile(csv_all_coins_day_full):
                            df_coin_day_all.to_csv(csv_all_coins_day_full, mode='w')
                        else:
                            df_coin_day_all.to_csv(csv_all_coins_day_full, mode='a', header=False)
            except Exception as e:
                logging.error(e)
                # logging.debug("Could not update data for {curr} from {exchange}".format(curr=symbol, exchange=exchange))
                not_updated[exchange].append(symbol)

logging.error("Did not update the following. Try again.\n {not_updated}".format(not_updated=not_updated))

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:21: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
ERROR:root:'NoneType' object has no attribute 'empty'
ERROR:root:'NoneType' object has no attribute 'empty'
ERROR:root:Did not update the following. Try again.
 defaultdict(<class 'list'>, {'Cryptopia': ['XRP'], 'Kucoin': ['XRP']})


In [41]:
# Maps csv (future data objects) to period granularity
# If we store all data together in a single data source, we'll change this to a function which returns corresponding rows
data_csv_period_mapping = {
    "day": 'all_coins_day_full.csv',
    "hour": 'all_coins_hour_full.csv',
    "min": 'all_coins_min_full.csv'
}

In [40]:
def fetch_data_csv(coin_symbol=['BTC'], nrows=1, period='day'):
    """ Fetch data from csv for mentioned coins 
        This function fetches last nrows from csv corr to given period
    """
    period = period.lower()
    csv_filename = data_csv_period_mapping[period]
    df_csv = pd.read_csv(csv_filename, index_col=None)
    # print(coin_symbol)
    return_df = []
    for coin in coin_symbol:
        # Get rows where 'coin' is one of the coin_symbols; then extract last nrows
        req_data = df_csv[df_csv['coin'] == coin].iloc[-nrows:]
        if req_data.empty:
            continue
        req_columns = ['coin', 'time', 'open', 'high', 'low', 'close', 'volumeto']
        req_data = req_data[req_columns]
        return_df.append(req_data)
    return return_df
    # return pd.concat(return_df)

In [10]:
df_day_10 = fetch_data_csv(list(COIN_DB.Symbol), nrows=10, period=TIMEFRAME)

In [12]:
import talib.abstract as taa
inp_data_coin1 = df_day_10[0]
inp_data_coin2 = df_day_10[1]
inp_sma = taa.SMA(inp_data_coin1, timeperiod=3)
inp_sma_two = taa.SMA(inp_data_coin2, timeperiod=5)
inp_rsi = talib.RSI(inp_data_coin1.close.values, timeperiod=14)
inp_mfi = talib.MFI(inp_data_coin1.high.values, inp_data_coin1.low.values, inp_data_coin1.close.values, inp_data_coin1.volumeto.values, timeperiod=14)
inp_bbands = talib.BBANDS(inp_data_coin1.close.values, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

In [39]:
def update_indicator(csv_filename, indicator_data, indicator):
    """ Update the given csv_file with new column values for corr rows """
    df_csv = pd.read_csv(csv_filename, index_col=None)
    if indicator not in df_csv.columns:
        df_csv[indicator] = np.nan
    df_csv = df_csv.set_index(['coin', 'time'])
    indicator_data = indicator_data.set_index(['coin', 'time'])
    df_csv.update(indicator_data)
    df_csv.to_csv(csv_filename)

In [38]:
def fetch_last_day(coin):
    """ Get data for last day """
    df_coin_last_day = ccw.get_historical_price_last_day(COIN)
    df_coin_last_day['coin'] = coin
    last_entry = df_coin_last_day.iloc[-1:]
    return last_entry

In [37]:
def update_last_day(coin):
    """ Update csv for last day """
    record_last_day = fetch_last_day(COIN)
    csv_day = 'coin_day_full.csv'
    df_day = pd.read_csv(csv_day)
    record = df_day[df_day['timestamp'] == record_last_day.iloc[0]['timestamp']]
    if record.empty:
        record_last_day.to_csv(csv_day, mode='a', header=False)

In [36]:
function_period_mapping = {
    'day': ccw.get_historical_price_day,
    'hour': ccw.get_historical_price_hour,
    'minute': ccw.get_historical_price_minute
}

In [35]:
def fetch_data_api(coin=COIN, to_curr=CURR, nperiods=1, period='day'):
    """ Fetch data for coin over nperiods
        e.g. Get data for 'BTC' for past 12 hours in hours granularity
    """
    period = period.lower()
    func = function_period_mapping[period]
    coin_last_nperiods = func(
        coin=coin,
        to_curr=to_curr,
        limit=nperiods
    )
    return coin_last_nperiods.iloc[-int(nperiods):]

In [54]:
def update_csv_to_latest(period='day'):
    """ Update the csv for given period upto current time for coin """
    period = period.lower()
    csv_filename = data_csv_period_mapping[period]  # Get corr csv
    df_coin_period = pd.read_csv(csv_filename)  # , index_col=['coin', 'exchange']
    csv_column_order = df_coin_period.columns.tolist()
    df_coin_period = df_coin_period.set_index(keys=['coin', 'exchange'])
    
    lst_new_data = []
    PRINT_MSG = "{:15} {!s:20} {!s:>20} {:>10}"
    logging.info(PRINT_MSG.format("Exchange", "Last Updated Time", "Elapsed Time", "nPeriodsAgo"))
    coins_in_csv = list(df_coin_period.index.get_level_values(0).unique())
    for coin in coins_in_csv:
        df_coin_period_coin = df_coin_period.loc[coin]
        # Group by exchange, sort on timestamp, and get the 
        last_update = df_coin_period_coin.groupby('exchange', group_keys=False).apply(lambda c: c.sort_values(by='time').tail(1))
        logging.info("-" * 10 + " For coin - {}".format(coin))
        # print(last_update)
        for exchange in last_update.index.values:
            last_updated_time = last_update.loc[exchange]['time']
            try:
                elapsed_time = datetime.datetime.now() - datetime.datetime.strptime(last_updated_time, '%Y-%m-%d %H:%M:%S')
            except ValueError as e:
                logging.info("Failed to parse time {} for {}--{}".format(last_updated_time, coin, exchange))
                elapsed_time = datetime.datetime.now() - dateutil.parser.parse(last_updated_time)
            nperiods_ago = elapsed_time / datetime.timedelta(days=1 if period == 'day' else 0,
                                                             hours=1 if period == 'hour' else 0,
                                                             minutes=1 if period == 'minute' else 0,
                                                             seconds=1)
            nperiods_ago = np.floor(nperiods_ago)
            
            logging.info(PRINT_MSG.format(exchange, last_updated_time, elapsed_time, nperiods_ago))
            
            if nperiods_ago > 0:
                logging.info("Updating data for {coin}-{exchange} from {last_updated_time}".format(
                    coin=coin, exchange=exchange, last_updated_time=last_updated_time)
                )
                new_data_coin_period = fetch_data_api(
                    coin=coin,
                    nperiods=nperiods_ago,
                    period=period
                )
                new_data_coin_period['coin'] = coin
                new_data_coin_period['exchange'] = exchange
                lst_new_data.append(new_data_coin_period)
    
    if lst_new_data:
        df_new_data = pd.concat(lst_new_data)
        curr_columns = df_new_data.columns.tolist()
        column_order = [col for col in csv_column_order if col in curr_columns]
        df_new_data = df_new_data.reindex(columns=column_order)
        df_new_data.to_csv(csv_filename, mode='a', header=False)

In [56]:
update_csv_to_latest()

2018-03-08 06:53:58,775 - INFO - Exchange        Last Updated Time            Elapsed Time nPeriodsAgo
2018-03-08 06:53:58,787 - INFO - ---------- For coin - ETH
2018-03-08 06:53:58,789 - INFO - Cryptopia       2018-03-07 16:00:00       14:53:58.789656        0.0
2018-03-08 06:53:58,790 - INFO - HitBTC          2018-03-07 16:00:00       14:53:58.790655        0.0
2018-03-08 06:53:58,792 - INFO - Kucoin          2018-03-07 16:00:00       14:53:58.792662        0.0
2018-03-08 06:53:58,801 - INFO - ---------- For coin - XRP
2018-03-08 06:53:58,802 - INFO - Failed to parse time 2018-03-06 16:00 for XRP--HitBTC
2018-03-08 06:53:58,804 - INFO - HitBTC          2018-03-06 16:00     1 day, 14:53:58.804693        1.0
2018-03-08 06:53:58,805 - INFO - Updating data for XRP-HitBTC from 2018-03-06 16:00
